In [13]:
import openai

In [3]:
api_key = ''

In [19]:
import openai
openai.api_key = api_key
# client = OpenAI()

def get_response(prompt:str):
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
            ]
    )
    return completion.choices[0].message


In [20]:
get_response('Hi I am depressed')

ChatCompletionMessage(content="I'm sorry to hear that you're feeling depressed. It's important to remember that you're not alone and that help is available. Have you tried talking to a therapist or counselor about how you're feeling? Additionally, practicing self-care activities such as exercise, meditation, and spending time with loved ones can also help improve your mood. Remember, it's okay to not be okay and seeking help is a brave step towards feeling better.", role='assistant', function_call=None, tool_calls=None)

In [25]:
prompt = 'Translate the text that is delimited by triple backticks into Hindi ```తెలుగు లిపి```'
get_response(prompt)

ChatCompletionMessage(content='```हिंदी लिपि```', role='assistant', function_call=None, tool_calls=None)

In [6]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key= api_key,temperature = 0.0)

In [7]:
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000024438378A30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002443837A140>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [25]:
from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "{input}"),
    ]
)

In [26]:
prompt1 = chat_template.format_messages(name='Sita', input='Can you please help me with my Math homework. What the fraction 1/7 in decimals?')

In [27]:
prompt1

[SystemMessage(content='You are a helpful AI bot. Your name is Sita.'),
 HumanMessage(content='Can you please help me with my Math homework. What the fraction 1/7 in decimals?')]

In [28]:
chat(prompt1)

AIMessage(content='Of course! To convert the fraction 1/7 into a decimal, you simply divide 1 by 7. \n\n1 ÷ 7 = 0.142857...\n\nSo, the fraction 1/7 in decimal form is approximately 0.142857.', response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 44, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b300e69d-c011-48b9-a361-c9f76161db35-0', usage_metadata={'input_tokens': 44, 'output_tokens': 56, 'total_tokens': 100})

In [29]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [38]:
output_parser = ChatPromptTemplate.from_template(review_template)
output_parser

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [40]:
prompt2 = output_parser.format_messages(text=customer_review)

In [33]:
chat(prompt2)

AIMessage(content='{\n    "gift": true,\n    "delivery_days": 2,\n    "price_value": ["It\'s slightly more expensive than the other leaf blowers out there"]\n}', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 218, 'total_tokens': 254}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec5001eb-9d3e-4cf3-abf1-6c9a9ddcb61a-0', usage_metadata={'input_tokens': 218, 'output_tokens': 36, 'total_tokens': 254})

In [35]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [44]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [56]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [67]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [68]:
messages[0].content

'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "

In [69]:
response = chat(messages)

In [70]:
response.content

'```json\n{\n\t"gift": true,\n\t"delivery_days": 2,\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [71]:
parsed_response = output_parser.parse(response.content)

In [72]:
parsed_response

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [73]:
print(parsed_response['gift'])

True
